In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

import numpy as np



# df = pd.read_excel('test.xlsx')
df = pd.read_csv('data/ratings.csv')
# print(df)

# df2 = df[['ID','Product','Qty']]
# print(df2)

df2 = df[['userId','movieId','rating']]
print(df2)



        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
100831     610   166534     4.0
100832     610   168248     5.0
100833     610   168250     5.0
100834     610   168252     5.0
100835     610   170875     3.0

[100836 rows x 3 columns]


In [28]:
pivotDf = pd.pivot_table(df2,index = 'userId',columns = 'movieId',values = 'rating',aggfunc = 'mean').reset_index().fillna(0).drop('userId', axis=1)
pivotDf = pivotDf.T.dot(pivotDf)

cosScore = pd.DataFrame(cosine_similarity(pivotDf))
cosScore.index = pivotDf.index
cosScore.columns = np.array(pivotDf.index)
# print(cosScore)
df_pairs = cosScore.stack().reset_index()
df_pairs = df_pairs.rename(columns={'movieId': 'MovieId1', 'level_1': 'MovieId2', 0: 'Score'})
df_pairs.reset_index(drop=True, inplace=True, level=0)
df_pairs = df_pairs[df_pairs['Score'] > 0.2].sort_values('Score', ascending=False)

In [29]:
df_names = pd.read_csv('data/movies.csv')

df_p = df_pairs.copy()

df_p = df_p.merge(df_names, how='inner', left_on='MovieId1', right_on='movieId').rename(columns={'title':'movie_1_title','genres':'movie_1_genres'}).drop(columns='movieId')
df_p = df_p.merge(df_names, how='inner', left_on='MovieId2', right_on='movieId').rename(columns={'title':'movie_2_title','genres':'movie_2_genres'}).drop(columns='movieId')



print(df_p)

          MovieId1  MovieId2     Score                 movie_1_title          movie_1_genres        movie_2_title        movie_2_genres
0             3909      3909  1.000000           Woman on Top (2000)          Comedy|Romance  Woman on Top (2000)        Comedy|Romance
1             2203      3909  0.761385      Shadow of a Doubt (1943)    Crime|Drama|Thriller  Woman on Top (2000)        Comedy|Romance
2             6268      3909  0.806856  Raising Victor Vargas (2002)    Comedy|Drama|Romance  Woman on Top (2000)        Comedy|Romance
3              634      3909  0.355600           Theodore Rex (1995)                  Comedy  Woman on Top (2000)        Comedy|Romance
4             3102      3909  0.648235            Jagged Edge (1985)  Crime|Romance|Thriller  Woman on Top (2000)        Comedy|Romance
...            ...       ...       ...                           ...                     ...                  ...                   ...
80191709      3719      3567  0.447328   Love's 

In [14]:
print(df_names)

      movieId                                      title                                       genres
0           1                           Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
1           2                             Jumanji (1995)                   Adventure|Children|Fantasy
2           3                    Grumpier Old Men (1995)                               Comedy|Romance
3           4                   Waiting to Exhale (1995)                         Comedy|Drama|Romance
4           5         Father of the Bride Part II (1995)                                       Comedy
...       ...                                        ...                                          ...
9737   193581  Black Butler: Book of the Atlantic (2017)              Action|Animation|Comedy|Fantasy
9738   193583               No Game No Life: Zero (2017)                     Animation|Comedy|Fantasy
9739   193585                               Flint (2017)                          

In [30]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

print(df_p[df_p['MovieId1'] == 1].sort_values('Score', ascending=False).to_excel('result.xlsx'))

None
